In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import time
import math
import re
import hashlib
from datetime import datetime, timedelta, timezone
from typing import List, Tuple, Optional, Dict

import pandas as pd
import requests
from zoneinfo import ZoneInfo

# ========= Config base (mantiene la lógica de tu v4) =========
TZ_DEFAULT = "Europe/Madrid"

BINANCE_API = "https://api.binance.com"
ENDPOINTS = [
    f"{BINANCE_API}/api/v3/klines",
    "https://api1.binance.com/api/v3/klines",
    "https://data-api.binance.vision/api/v3/klines",
]
EXCHANGE_INFO = f"{BINANCE_API}/api/v3/exchangeInfo"
TICKER_24HR = f"{BINANCE_API}/api/v3/ticker/24hr"

INTERVAL_OPTIONS = ["1m","3m","5m","15m","30m","1h","2h","4h","6h","12h","1d"]

# Nombres legibles (opcional)
NAME_HINTS: Dict[str,str] = {
    "BTC":"Bitcoin","ETH":"Ethereum","BNB":"BNB","XRP":"XRP","ADA":"Cardano","SOL":"Solana",
    "DOGE":"Dogecoin","TRX":"TRON","TON":"Toncoin","DOT":"Polkadot","AVAX":"Avalanche",
    "LINK":"Chainlink","MATIC":"Polygon","LTC":"Litecoin","BCH":"Bitcoin Cash","ATOM":"Cosmos",
    "ETC":"Ethereum Classic","XLM":"Stellar","ICP":"Internet Computer","APT":"Aptos","ARB":"Arbitrum",
    "OP":"Optimism","NEAR":"NEAR","SUI":"Sui","FTM":"Fantom","AAVE":"Aave","UNI":"Uniswap",
}

def log(msg: str): print(f"[INFO] {msg}", flush=True)
def warn(msg: str): print(f"[WARN] {msg}", flush=True)
def err(msg: str): print(f"[ERROR] {msg}", file=sys.stderr, flush=True)

# ========= Ventana 1 año con fin exclusivo (v4) =========
def compute_window_ms(tz_name: str, days: int) -> Tuple[int, int, int, datetime, datetime]:
    """
    Ventana:
      start_ms            -> (ayer - (days-1)) 00:00 local (incluido)
      end_ms (informativo)-> ayer 23:59:59 local
      end_ms_exclusive    -> hoy 00:00 local (exclusivo) => incluye la vela final de ayer
    """
    tz = ZoneInfo(tz_name)
    now_tz = datetime.now(tz)
    yesterday = now_tz.date() - timedelta(days=1)

    end_local = datetime.combine(yesterday, datetime.max.time()).replace(microsecond=0, tzinfo=tz)
    start_local = (end_local - timedelta(days=days - 1)).replace(hour=0, minute=0, second=0, microsecond=0)
    end_local_exclusive = datetime.combine(yesterday + timedelta(days=1), datetime.min.time()).replace(tzinfo=tz)

    start_utc = start_local.astimezone(timezone.utc)
    end_utc = end_local.astimezone(timezone.utc)
    end_utc_exclusive = end_local_exclusive.astimezone(timezone.utc)

    return (
        int(start_utc.timestamp() * 1000),
        int(end_utc.timestamp() * 1000),
        int(end_utc_exclusive.timestamp() * 1000),
        start_utc, end_utc
    )

# ========= Requests con reintentos (v4) =========
def get_with_retries(url: str, params: dict = None, timeout_connect=5, timeout_read=20, retries=4, debug=False):
    params = params or {}
    backoff = 2
    for attempt in range(1, retries + 1):
        try:
            if debug: log(f"-> GET {url} (attempt {attempt}) params={params}")
            r = requests.get(url, params=params, timeout=(timeout_connect, timeout_read))
            if r.status_code == 200:
                return r
            warn(f"HTTP {r.status_code}: {r.text[:200]}")
            if r.status_code in (429, 418, 500, 502, 503, 504):
                time.sleep(backoff); backoff = min(backoff * 2, 30); continue
            r.raise_for_status()
        except (requests.Timeout, requests.ConnectionError) as e:
            warn(f"Conexion/timeout: {e}. Reintentando en {backoff}s...")
            time.sleep(backoff); backoff = min(backoff * 2, 30)
        except Exception:
            raise
    raise TimeoutError(f"No se obtuvo respuesta OK tras {retries} intentos a {url}")

# ========= Descarga de klines (v4) =========
def fetch_binance_klines(symbol: str, start_ms: int, end_ms_exclusive: int, interval="1h", debug=False) -> List[list]:
    """
    Pagina hasta cubrir TODO el rango [start_ms, end_ms_exclusive).
    No corta por len(batch)<limit (Binance a veces devuelve menos).
    """
    limit = 1500
    all_rows: List[list] = []
    current_start = start_ms
    last_progress = None

    while current_start < end_ms_exclusive:
        params = {
            "symbol": symbol.upper(),
            "interval": interval,
            "startTime": current_start,
            "endTime": end_ms_exclusive,  # EXCLUSIVO
            "limit": limit
        }

        resp = None; last_exc = None
        for base in ENDPOINTS:
            try:
                resp = get_with_retries(base, params, timeout_connect=5, timeout_read=20, retries=4, debug=debug)
                break
            except Exception as e:
                last_exc = e; warn(f"{symbol}: fallo en {base.split('//')[-1]} -> {e}")

        if resp is None:
            raise RuntimeError(f"{symbol}: todos los endpoints fallaron: {last_exc}")

        batch = resp.json()
        if not isinstance(batch, list):
            raise ValueError(f"{symbol}: respuesta inesperada: {batch}")
        if not batch:
            warn(f"{symbol}: batch vacio desde start={current_start}. Deteniendo.")
            break

        all_rows.extend(batch)

        first_dt = pd.to_datetime(batch[0][0], unit="ms", utc=True).tz_convert(TZ_DEFAULT)
        last_dt = pd.to_datetime(batch[-1][0], unit="ms", utc=True).tz_convert(TZ_DEFAULT)
        log(f"{symbol}: +{len(batch)} velas ({first_dt:%Y-%m-%d %H:%M} -> {last_dt:%Y-%m-%d %H:%M} {TZ_DEFAULT}) total={len(all_rows)}")

        # Avanzar (closeTime + 1ms)
        last_close_time = int(batch[-1][6])
        next_start = last_close_time + 1

        # Seguridad: asegurar progreso
        if last_progress is not None and next_start <= last_progress:
            warn(f"{symbol}: no hay progreso (next_start <= last_progress). Deteniendo para evitar bucle.")
            break
        last_progress = next_start

        current_start = next_start
        time.sleep(0.12)

    return all_rows

# ========= Transformación a un CSV único con “máx columnas” =========
def klines_to_df_full(klines: List[list], tz_name: str, symbol: str, interval: str) -> pd.DataFrame:
    """
    Columnas incluidas:
      - ts_open_local, ts_close_local, ts_open_utc, ts_close_utc
      - symbol, interval
      - open, high, low, close
      - volume_base, volume_quote, trades
      - taker_buy_base, taker_buy_quote
      - vwap_usdt (approx = volume_quote / volume_base si >0)
      - spread_abs, spread_pct
      - return_close_pct (vs cierre previo)
      - dow (0=lun..6=dom), hour_local
    """
    cols = ["ts_open_local","ts_close_local","ts_open_utc","ts_close_utc",
            "symbol","interval",
            "open","high","low","close",
            "volume_base","volume_quote","trades",
            "taker_buy_base","taker_buy_quote",
            "vwap_usdt","spread_abs","spread_pct",
            "return_close_pct","dow","hour_local"]
    if not klines:
        return pd.DataFrame(columns=cols)

    tz = ZoneInfo(tz_name)
    rows = []
    for k in klines:
        # kline:
        # 0 openTime, 1 open, 2 high, 3 low, 4 close, 5 volume_base,
        # 6 closeTime, 7 quoteAssetVolume, 8 trades,
        # 9 takerBuyBaseVol, 10 takerBuyQuoteVol, 11 ignore
        open_ms = int(k[0]); close_ms = int(k[6])
        open_utc = datetime.fromtimestamp(open_ms/1000, tz=timezone.utc)
        close_utc = datetime.fromtimestamp(close_ms/1000, tz=timezone.utc)
        open_local = open_utc.astimezone(tz)
        close_local = close_utc.astimezone(tz)

        o = float(k[1]); h = float(k[2]); l = float(k[3]); c = float(k[4])
        vb = float(k[5]); vq = float(k[7]); tr = int(k[8])
        tb = float(k[9]); tq = float(k[10])

        vwap = (vq / vb) if vb > 0 else math.nan
        spread_abs = (h - l)
        spread_pct = ((h / l) - 1.0) if l > 0 else math.nan

        rows.append((
            open_local, close_local, open_utc, close_utc,
            symbol.upper(), interval,
            o, h, l, c, vb, vq, tr, tb, tq, vwap, spread_abs, spread_pct,
            math.nan, open_local.weekday(), open_local.hour
        ))

    df = pd.DataFrame(rows, columns=cols)
    df = (df.drop_duplicates(subset=["symbol","ts_open_local"])
            .sort_values(["symbol","ts_open_local"])
            .reset_index(drop=True))
    df["return_close_pct"] = df.groupby("symbol")["close"].pct_change()
    return df

def trim_to_local_window(df: pd.DataFrame, tz_name: str, start_ms: int, end_ms_exclusive: int) -> pd.DataFrame:
    tz = ZoneInfo(tz_name)
    start_local = datetime.fromtimestamp(start_ms/1000, tz=timezone.utc).astimezone(tz)
    end_local_excl = datetime.fromtimestamp(end_ms_exclusive/1000, tz=timezone.utc).astimezone(tz)
    return df[(df["ts_open_local"] >= start_local) & (df["ts_open_local"] < end_local_excl)].copy()

def sha256_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

# ========= Menú A: Top-25 contra USDT desde Binance (sin API keys) =========
def get_top25_usdt_from_binance(debug=False) -> List[dict]:
    # 1) exchangeInfo para validar pares (estado TRADING y quote USDT)
    ex = get_with_retries(EXCHANGE_INFO, retries=3, timeout_read=25, debug=debug).json()
    symbol_map = {}
    for s in ex.get("symbols", []):
        symbol_map[s["symbol"]] = {
            "base": s.get("baseAsset"),
            "quote": s.get("quoteAsset"),
            "status": s.get("status")
        }
    # 2) ticker/24hr para volúmenes
    t = get_with_retries(TICKER_24HR, retries=3, timeout_read=25, debug=debug).json()
    rows = []
    for it in t:
        sym = it.get("symbol")
        info = symbol_map.get(sym)
        if not info: continue
        if info["quote"] != "USDT": continue
        if info["status"] != "TRADING": continue
        # Excluir tokens apalancados/sintéticos UP/DOWN/BULL/BEAR
        if re.search(r"(UP|DOWN|BULL|BEAR)$", sym): continue
        try:
            quote_vol = float(it.get("quoteVolume", 0.0))
            base = info["base"]
        except Exception:
            continue
        rows.append((sym, base, quote_vol))
    rows.sort(key=lambda x: x[2], reverse=True)
    top = rows[:25]
    out = []
    for sym, base, vol in top:
        out.append({"symbol": sym, "base": base, "name": NAME_HINTS.get(base, base), "quoteVolume": vol})
    return out

def pick_from_menu(toplist: List[dict]) -> dict:
    print("\n=== Top 25 USDT por volumen 24h (Binance) ===")
    for i, item in enumerate(toplist, start=1):
        print(f"{i:2d}) {item['name']} ({item['base']}USDT)")
    while True:
        sel = input("Elige numero (1-25): ").strip()
        if sel.isdigit() and 1 <= int(sel) <= len(toplist):
            return toplist[int(sel)-1]
        print("Seleccion invalida.")

# ========= Menú B: Intervalo =========
def pick_interval_menu() -> str:
    print("\n=== Intervalo (incluye granularidades 1m–1d) ===")
    for i, iv in enumerate(INTERVAL_OPTIONS, start=1):
        print(f"{i:2d}) {iv}")
    while True:
        sel = input("Elige numero: ").strip()
        if sel.isdigit() and 1 <= int(sel) <= len(INTERVAL_OPTIONS):
            return INTERVAL_OPTIONS[int(sel)-1]
        print("Seleccion invalida.")

# ========= Main interactivo (un solo archivo de salida) =========
def main():
    log(f"Zona horaria base: {TZ_DEFAULT}")

    # Menú A
    try:
        top = get_top25_usdt_from_binance(debug=False)
    except Exception as e:
        err(f"No se pudo construir el Top-25 desde Binance: {e}")
        sys.exit(1)
    pick = pick_from_menu(top)
    base = pick["base"]
    symbol = f"{base}USDT"
    human = pick["name"]
    log(f"Seleccion: {human} ({symbol})")

    # Menú B
    interval = pick_interval_menu()
    log(f"Intervalo elegido: {interval}")

    # Ventana 1 año hasta ayer (fin exclusivo hoy 00:00)
    start_ms, end_ms, end_ms_exclusive, start_utc, end_utc = compute_window_ms(TZ_DEFAULT, 2)
    log(f"Ventana local: {start_utc.astimezone(ZoneInfo(TZ_DEFAULT)):%Y-%m-%d %H:%M} -> {end_utc.astimezone(ZoneInfo(TZ_DEFAULT)):%Y-%m-%d %H:%M}")
    log(f"(UTC ms) start={start_ms}, end_informativo={end_ms}, end_exclusivo={end_ms_exclusive}")

    # Descarga
    try:
        kl = fetch_binance_klines(symbol, start_ms, end_ms_exclusive, interval=interval, debug=False)
    except Exception as e:
        err(f"{symbol}: fallo al descargar -> {e}")
        sys.exit(1)

    if not kl:
        err("No se recibieron klines. Revisa conectividad o rango.")
        sys.exit(1)

    # Transformación a DF con “máx columnas”
    df = klines_to_df_full(kl, TZ_DEFAULT, symbol, interval)
    df = trim_to_local_window(df, TZ_DEFAULT, start_ms, end_ms_exclusive)

    # Salida única
    outdir = "."
    os.makedirs(outdir, exist_ok=True)
    outname = os.path.join(outdir, f"{symbol.lower()}_{interval}_1y_full.csv")
    df.to_csv(outname, index=False, date_format="%Y-%m-%d %H:%M:%S%z")
    log(f"CSV guardado: {outname} (sha256 {sha256_file(outname)})")

    if not df.empty:
        log(f"Rango final: {df['ts_open_local'].min()} -> {df['ts_open_local'].max()} | filas={len(df)}")
    print("Columnas:", ", ".join(df.columns))
    log("✅ Listo.")

if __name__ == "__main__":
    main()

[INFO] Zona horaria base: Europe/Madrid

=== Top 25 USDT por volumen 24h (Binance) ===
 1) USDC (USDCUSDT)
 2) Bitcoin (BTCUSDT)
 3) Ethereum (ETHUSDT)
 4) Solana (SOLUSDT)
 5) ENSO (ENSOUSDT)
 6) BNB (BNBUSDT)
 7) FDUSD (FDUSDUSDT)
 8) XRP (XRPUSDT)
 9) TRUMP (TRUMPUSDT)
10) ZEC (ZECUSDT)
11) HBAR (HBARUSDT)
12) Dogecoin (DOGEUSDT)
13) EUL (EULUSDT)
14) TAO (TAOUSDT)
15) ASTER (ASTERUSDT)
16) Litecoin (LTCUSDT)
17) Sui (SUIUSDT)
18) PUMP (PUMPUSDT)
19) ZBT (ZBTUSDT)
20) VIRTUAL (VIRTUALUSDT)
21) USDE (USDEUSDT)
22) Bitcoin Cash (BCHUSDT)
23) PAXG (PAXGUSDT)
24) TRON (TRXUSDT)
25) Cardano (ADAUSDT)


Elige numero (1-25):  2


[INFO] Seleccion: Bitcoin (BTCUSDT)

=== Intervalo (incluye granularidades 1m–1d) ===
 1) 1m
 2) 3m
 3) 5m
 4) 15m
 5) 30m
 6) 1h
 7) 2h
 8) 4h
 9) 6h
10) 12h
11) 1d


Elige numero:  1


[INFO] Intervalo elegido: 1m
[INFO] Ventana local: 2025-10-26 00:00 -> 2025-10-27 23:59
[INFO] (UTC ms) start=1761429600000, end_informativo=1761605999000, end_exclusivo=1761606000000
[INFO] BTCUSDT: +1000 velas (2025-10-26 00:00 -> 2025-10-26 15:39 Europe/Madrid) total=1000
[INFO] BTCUSDT: +1000 velas (2025-10-26 15:40 -> 2025-10-27 08:19 Europe/Madrid) total=2000
[INFO] BTCUSDT: +941 velas (2025-10-27 08:20 -> 2025-10-28 00:00 Europe/Madrid) total=2941
[INFO] CSV guardado: .\btcusdt_1m_1y_full.csv (sha256 275cca25661f0ec5b02718d35788a4f7b95db8e1cefe9ffb32d6410f31444c04)
[INFO] Rango final: 2025-10-26 00:00:00+02:00 -> 2025-10-27 23:59:00+01:00 | filas=2940
Columnas: ts_open_local, ts_close_local, ts_open_utc, ts_close_utc, symbol, interval, open, high, low, close, volume_base, volume_quote, trades, taker_buy_base, taker_buy_quote, vwap_usdt, spread_abs, spread_pct, return_close_pct, dow, hour_local
[INFO] ✅ Listo.
